In [2]:
import numpy as np
import cv2

In [3]:
# največji skupni deljitel
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

# najmanjši skupni večkratnik
def lcm(a, b):
    return abs(a * b) // gcd(a, b)

# najmanši skupni večkratnik večih števil
def smallest_common_multiple(numbers):
    result = 1
    for num in numbers:
        result = lcm(result, num)
    return result

class SimpleSensor:
    def __init__(self, spaceDim, start, stop): # start stop naj bosta [*, 0], [*, 0] ali [0, *], [0, *]
        self.start = np.array(start)
        self.stop = np.array(stop)
        self.spaceDim = spaceDim
        self.timePeriod = 2*np.max(np.abs(self.start - self.stop))

    def getSensorArea(self):
        if self.start[0] == self.stop[0]:
            direction = np.array([0, 1])*(((self.start[1] - self.stop[1])<0)*2-1)
        elif self.start[1] == self.stop[1]:
            direction = np.array([1, 0])*(((self.start[0] - self.stop[0])<0)*2-1)
        space = np.zeros((self.timePeriod, *self.spaceDim))
        rangee = int(self.timePeriod/2)
        for t in range(rangee+1):
            y, x = self.start + t*direction
            space[t][y][x] = 1
            space[t][y][x+1] = 1
            space[t][y+1][x] = 1
            space[t][y+1][x+1] = 1
        for t in range(0, rangee-1):
            y, x = self.stop - (t+1)*direction
            space[rangee+1 + t][y][x] = 1
            space[rangee+1 + t][y][x+1] = 1
            space[rangee+1 + t][y+1][x] = 1
            space[rangee+1 + t][y+1][x+1] = 1
        return space

# Vzame seznam začetekov in koncev senzorjev ter jih združi v en prostor
class SensorsSpace:
    def __init__(self, spaceDim, sensorStartStopList):
        self.sensorStartStopList = sensorStartStopList # [[[startX,startY],[stopX,*]], [[*,*],[*,*]], [[*,*],[*,*]] ]
        self.spaceDim = spaceDim #[*,*]
        self.sensors = []
        self.getCommonTimePeriod()

    def getCommonTimePeriod(self):
        periods = []
        for startStop in self.sensorStartStopList:
            sensor = SimpleSensor(self.spaceDim, startStop[0], startStop[1])
            self.sensors.append(sensor)
            periods.append(sensor.timePeriod)

        self.timePeriod = smallest_common_multiple(periods)

    def createSpace(self):
        space = np.zeros((self.timePeriod, *self.spaceDim))
        for sensor in self.sensors:
            n_roundtrips = int(self.timePeriod/sensor.timePeriod)
            
            space += np.tile(sensor.getSensorArea(), (n_roundtrips, 1, 1))
        return space > 0

In [26]:
sensors = [[[0,8],[0,18]], [[2,0],[18,0]], [[15,2],[15,12]]]
sensorSpace = SensorsSpace((20,20), sensors)
space = sensorSpace.createSpace()
print(len(space))

160


In [27]:
video = space.astype(float)

n_frames, height, width = np.shape(video)
newWidth = 800
newHeight = int(height/width*newWidth)

newVideo = []
for f in range(n_frames):
    newImage = np.reshape(video[f], (height, width, 1))
    newImage = cv2.resize(newImage, (newWidth, newHeight))
    newImage[newImage>0] = 1
    newImage[0,:] = 1
    newImage[-1,:] = 1
    newImage[:,0] = 1
    newImage[:,-1] = 1
    newImage *= 255
    
    newVideo.append(newImage)
newVideo = np.array(newVideo, dtype=np.uint8)
newShape = np.shape(newVideo)
# cv2.imshow('Image', newVideo[1])

# # Wait for a key event and close the window when a key is pressed
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [28]:
import cv2
import numpy as np

# Specify the video file name, codec, frames per second (fps), and resolution
video_file = 'output_video.avi'
codec = cv2.VideoWriter_fourcc(*'XVID')
fps = 24.0
resolution = (newShape[2], newShape[1])

# Create a VideoWriter object
video_writer = cv2.VideoWriter(video_file, codec, fps, resolution)

videoSlower = 5
# Generate frames (you can replace this with your own frames)
for i in range(videoSlower*len(newVideo)):
    #frame = np.ones((resolution[1], resolution[0], 3), dtype=np.uint8) * 255
    # cv2.putText(frame, f'Frame {i}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    frame = np.repeat(newVideo[i//videoSlower][:, :, np.newaxis], 3, axis=2)
    # print(np.shape(frame1))
    # Write the frame to the video file
    video_writer.write(frame)

# Release the VideoWriter
video_writer.release()

print(f"Video created: {video_file}")


Video created: output_video.avi
